# 1. Extract data from csv file and then serialize

In [8]:
import pickle
import csv

In [49]:
ls

les.pickle        solution.ipynb    store_states.csv* train.pickle
models.py*        store.csv         test.csv          train_XY.pickle
models.pyc        store.pickle      train.csv


In [7]:
def csvToDict(csvfile):
    datas = []
    keys = []
    for row_index, row in enumerate(csvfile):
        if row_index == 0:
            keys = row
            print keys
            continue
        datas.append({key : val for key, val in zip(keys, row)}) # each element is a sample in a dict format
    return datas

In [10]:
def setNanAsString(datas, default_str = '0'):
    for i, sample in enumerate(datas):
        for key, val in sample.items():
            if val == '':
                sample[key] = default_str
        datas[i] = sample   

In [12]:
train_data = "train.csv"
store_data = "store.csv"
store_states_data = "store_states.csv"

In [13]:
with open(train_data) as csvfile:
    csv_data = csv.reader(csvfile, delimiter = ',')
    with open("train.pickle","wb") as dump_file:
        datas = csvToDict(csv_data)
        datas = datas[::-1]
        pickle.dump(datas, dump_file, -1)
        print datas[:3]

['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
[{'DayOfWeek': '2', 'Customers': '0', 'StateHoliday': 'a', 'Promo': '0', 'Sales': '0', 'Date': '2013-01-01', 'Open': '0', 'SchoolHoliday': '1', 'Store': '1115'}, {'DayOfWeek': '2', 'Customers': '0', 'StateHoliday': 'a', 'Promo': '0', 'Sales': '0', 'Date': '2013-01-01', 'Open': '0', 'SchoolHoliday': '1', 'Store': '1114'}, {'DayOfWeek': '2', 'Customers': '0', 'StateHoliday': 'a', 'Promo': '0', 'Sales': '0', 'Date': '2013-01-01', 'Open': '0', 'SchoolHoliday': '1', 'Store': '1113'}]


In [18]:
with open(store_data) as csvfile1, open(store_states_data) as csvfile2:
    csv_data1 = csv.reader(csvfile1, delimiter = ',')
    csv_data2 = csv.reader(csvfile2, delimiter = ',')
    with open("store.pickle","wb") as dump_file:
        store_datas = csvToDict(csv_data1)
        store_states_datas = csvToDict(csv_data2)
        setNanAsString(store_datas)
        for i, store in enumerate(store_datas):
            store["State"] = store_states_datas[i]["State"]
            store_datas[i] = store
        pickle.dump(store_datas, dump_file, -1)
        print store_datas[:2]

['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
['Store', 'State']
[{'CompetitionOpenSinceMonth': '9', 'Promo2SinceYear': '0', 'Promo2SinceWeek': '0', 'CompetitionDistance': '1270', 'PromoInterval': '0', 'State': 'HE', 'Promo2': '0', 'CompetitionOpenSinceYear': '2008', 'Assortment': 'a', 'StoreType': 'c', 'Store': '1'}, {'CompetitionOpenSinceMonth': '11', 'Promo2SinceYear': '2010', 'Promo2SinceWeek': '13', 'CompetitionDistance': '570', 'PromoInterval': 'Jan,Apr,Jul,Oct', 'State': 'TH', 'Promo2': '1', 'CompetitionOpenSinceYear': '2007', 'Assortment': 'a', 'StoreType': 'a', 'Store': '2'}]


# 2. Prepare feature

In [20]:
from datetime import datetime
from sklearn import preprocessing
import numpy as np
import random
import pickle
random.seed(42)

In [21]:
with open("train.pickle") as f:
    train_data = pickle.load(f)
with open("store.pickle") as f:
    store_data = pickle.load(f)

In [22]:
def getCleanFeature(sample):
    dt = datetime.strptime(sample["Date"], "%Y-%m-%d")
    store_index = int(sample["Store"])
    year = dt.year
    month = dt.month
    day = dt.day
    day_of_week = int(sample["DayOfWeek"])
    try:
        store_open = int(sample['Open'])
    except:
        store_open = 1
    promo = int(sample['Promo'])    
    return [    
                store_open,
                store_index,
                day_of_week,
                promo,
                year,
                month,
                day,
                store_data[store_index - 1]['State']
            ]    

In [23]:
train_X = []
train_Y = []

In [28]:
for sample in train_data:
    if sample["Sales"] != "0" and sample["Open"] != "":
        x = getCleanFeature(sample)
        train_X.append(x)
        train_Y.append(int(sample["Sales"]))
print "number of train datapoints: " + str(len(train_Y))
print "sales turnover range from: " + str(min(train_Y)) + " to " + str(max(train_Y))

number of train datapoints: 4221690
sales turnover range from: 46 to 41551


In [42]:
train_X = np.array(train_X)
train_Y = np.array(train_Y)
full_X = np.array(train_X)

les = []
for col in range(train_X.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(full_X[:, col])
    les.append(le)
    train_X[:, col] = le.transform(train_X[:, col])
train_X = train_X.astype(int)
    
with open('les.pickle', 'wb') as f:
    pickle.dump(les, f, -1)

with open('train_XY.pickle', 'wb') as f:
    pickle.dump((train_X, train_Y), f, -1)
    print(train_X[0], train_Y[0])

(array([  0, 109,   1,   0,   0,   0,   0,   7]), 5961)


# 3. Model

In [46]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.preprocessing import Normalizer

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Reshape
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint

import numpy
import xgboost as xgb
import pickle

numpy.random.seed(123)

In [99]:
def split_features(X):
    X_list = []

    store_index = X[..., [1]]
    X_list.append(store_index)

    day_of_week = X[..., [2]]
    X_list.append(day_of_week)

    promo = X[..., [3]]
    X_list.append(promo)

    year = X[..., [4]]
    X_list.append(year)

    month = X[..., [5]]
    X_list.append(month)

    day = X[..., [6]]
    X_list.append(day)

    State = X[..., [7]]
    X_list.append(State)

    return X_list

In [273]:
class NN_with_EntityEmbedding(object):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.nb_epoch = 1
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(numpy.max(numpy.log(y_train)), numpy.max(numpy.log(y_val)))
        self.__build_keras_model()
    
    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list    
    
    def __build_keras_model(self):
        models = []

        model_store = Sequential()
        model_store.add(Embedding(1115, 10, input_length=1))
        model_store.add(Reshape(target_shape=(10,)))
        models.append(model_store)

        model_dow = Sequential()
        model_dow.add(Embedding(7, 6, input_length=1))
        model_dow.add(Reshape(target_shape=(6,)))
        models.append(model_dow)

        model_promo = Sequential()
        model_promo.add(Dense(1, input_dim=1))
        models.append(model_promo)

        model_year = Sequential()
        model_year.add(Embedding(3, 2, input_length=1))
        model_year.add(Reshape(target_shape=(2,)))
        models.append(model_year)

        model_month = Sequential()
        model_month.add(Embedding(12, 6, input_length=1))
        model_month.add(Reshape(target_shape=(6,)))
        models.append(model_month)

        model_day = Sequential()
        model_day.add(Embedding(31, 10, input_length=1))
        model_day.add(Reshape(target_shape=(10,)))
        models.append(model_day)

        model_germanstate = Sequential()
        model_germanstate.add(Embedding(12, 6, input_length=1))
        model_germanstate.add(Reshape(target_shape=(6,)))
        models.append(model_germanstate)

        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(1000, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(500, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = numpy.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return numpy.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train),
                       validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)),
                       nb_epoch=self.nb_epoch, batch_size=128,
                       )
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)
    
    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0)
        guessed_sales = self.guess(X_val)
        relative_err = numpy.absolute((y_val - guessed_sales) / y_val)
        result = numpy.sum(relative_err) / len(y_val)
        return result    
    

# 4. Run

In [44]:
import pickle
import numpy
from models import *
from sklearn.preprocessing import OneHotEncoder
import sys
sys.setrecursionlimit(10000)
numpy.random.seed(123)

Using Theano backend.


In [150]:
train_ratio = 0.9
shuffle_data = False
one_hot_as_input = False
embeddings_as_input = False
save_embeddings = True
save_models = True
saved_embeddings_fname = "embeddings.pickle"

In [51]:
(X, y) = pickle.load(open("train_XY.pickle", "rb"))
num_records = len(X)
train_size = int(train_ratio * num_records)

In [65]:
if shuffle_data:
    print("using shuffled data")
    sh = numpy.arange(X.shape[0])
    numpy.random.shuffle(sh)
    X = X[sh]
    y = y[sh]

In [89]:
if one_hot_as_input:
    print("using one-hot encoding as input")
    enc = OneHotEncoder(sparse=False)
    enc.fit(X)
    X = enc.transform(X)

In [90]:
def sample(X, y, n):
    num_records = X.shape[0]
    indices = numpy.random.randint(num_records, size=n)
    return X[indices, :], y[indices]

In [88]:
X_train = X[:train_size]
X_val = X[train_size:]
y_train = y[:train_size]
y_val = y[train_size:]

In [141]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("number of samples used for training: " + str(y_train.shape[0]))

number of samples used for training: 200000


In [294]:
print("fitting NN_with_EntityEmbedding...")
NNEE = NN_with_EntityEmbedding(X_train, y_train, X_val, y_val)
NNEE.fit(X_train, y_train, X_val, y_val)

fitting NN_with_EntityEmbedding...
Train on 200000 samples, validate on 422169 samples
Epoch 1/1
200000/200000 [==============================] - 73s - loss: 0.0142 - val_loss: 0.0098
('Result on validation data: ', 0.10834435437077393)


In [326]:
weights = []
merged_layer = NNEE.model.layers[0]
for embedding_layer in merged_layer.layers:
    weights.append(embedding_layer.get_weights()[0])
with open(saved_embeddings_fname, 'wb') as f:
    pickle.dump(weights, f, -1)

In [132]:
if save_models:
    with open('models.pickle', 'wb') as f:
        pickle.dump(models, f)

In [133]:
def evaluate_models(models, X, y):
    assert(min(y) > 0)
    guessed_sales = numpy.array([model.guess(X) for model in models])
    mean_sales = guessed_sales.mean(axis=0)
    relative_err = numpy.absolute((y - mean_sales) / y)
    result = numpy.sum(relative_err) / len(y)
    return result

In [134]:
print("Evaluate combined models...")
print("Training error...")
r_train = evaluate_models(models, X_train, y_train)
print(r_train)

print("Validation error...")
r_val = evaluate_models(models, X_val, y_val)
print(r_val)

Evaluate combined models...
Training error...
0.0782901170852
Validation error...
0.0808420976915
